In [57]:
import scipy
import pandas as pd
import numpy as np
import glob
import statistics
from statistics import mean

pd.set_option('display.max_columns', 500)

import warnings
warnings.simplefilter(action='ignore', category=Warning)

# описание папок с файлами и особенностями

NBA1947_1981 - файлы по 1 шт на сезон, исключая сезоны объединенной лиги с 1968 по 1976 - ТИП 1

ABA1968_1976 - по два файла на сезон из NBA и ABA / считаем среднее по всем матчам - ТИП 2

NBA1982 - в файле фигуригуют значения DNP и Do Not Play почти во всех столбцах - ТИП 3

NBA1983_1996 - обычные файлы - ТИП 1

NBA1997_2023 - очков в 3 раза больше, т.к. начали фиксировать в доп.столбце "Period" принадлежность к половине (h1/h2) и к четвертям (q1/q2/q3/q4), поэтому очков суммарно в 3 раза больше - ТИП 4

In [58]:
# СЧИТАЕМ СРЕДНЕЕ КОЛИЧЕСТВО ОЧКОВ ПО КАЖДОМУ МАТЧУ / для NBA1947_1981

total_df = pd.DataFrame(columns=['Year', 'PTS_mean'])       # СОЗДАДИМ ПУСТОЙ ДФ

for file in glob.glob('D:/KAGGLE/Datasets/NBA_1947_2024/NBA/boxscores_by_year/NBA1947_1981/*_basic.csv'):  # ЦИКЛ ДЛЯ ВСЕХ ФАЙЛОВ ВНУТРИ ПАПКИ
    
    my_df = pd.read_csv(file, engine='python')              # СЧИТЫВАЕМ В ПЕРЕМЕННУЮ 
   
    filename = file[77] + file[78] + file[79] + file[80]    # СОЗДАЕМ ПЕРЕМЕННУЮ ИЗ ИМЕНИ ФАЙЛА - НАЗВАНИЕ ГОДА ОКОНЧАНИЯ СЕЗОНА
    oppo = my_df.groupby('Game Reference').sum()            # СУММИРУЕМ ОЧКИ, ГРУППИРУЯ ПО КАЖДОМУ МАТЧУ
    oppo2 = oppo['PTS'].mean()                              # ИЩЕМ СРЕДНЕЕ ПО СЕЗОНУ
    
    total_df.loc[len(total_df.index)] = [filename, oppo2]   # СОХРАНЯЕМ В ДФ
   


In [80]:
# НАЙДЕМ СРЕДНЕЕ ДЛЯ ГОДОВ С ДВУМЯ ЛИГАМИ 1968 - 1976 - ВРУЧНУЮ МЕНЯЕМ КАЖДЫЙ ГОД, ЧТОБЫ ОБСЧИТЫВАЛ НЕСКОЛЬКО ФАЙЛОВ ОДНОГО ГОДА

sum_pts = 0
count_game = 0

for file in glob.glob('D:/KAGGLE/Datasets/NBA_1947_2024/NBA/boxscores_by_year/ABA1968_1976/*1976_basic.csv'):  
  
    
    my_df = pd.read_csv(file, engine='python')
    oppo = my_df.groupby('Game Reference', as_index=False).sum()   # СУММИРУЕМ ОЧКИ ПО КАЖДОМУ МАТЧУ
    count_game += oppo.shape[0]                                    # СОХРАНИМ КОЛИЧЕСТВО ИГР В ПЕРЕМЕННУЮ count_game
    sum_pts += oppo['PTS'].sum()                                   # СОХРАНИМ КОЛИЧЕСТВО  

answer = sum_pts / count_game                                      # НАХОДИМ СРЕДНЕЕ В ДВУХ ЛИГАХ

filename = file[77] + file[78] + file[79] + file[80]               # СОЗДАЕМ ПЕРЕМЕННУЮ НАЗВАНИЕ ГОДА ОКОНЧАНИЯ СЕЗОНА

total_df.loc[len(total_df.index)] = [filename, answer]             # СОХРАНЯЕМ В ДФ

In [186]:
# НАЙДЕМ СРЕДНЕЕ КОЛИЧЕСТВО ОЧКОВ ТОЛЬКО ДЛЯ 1982 . 
# ПРЕДЫДУЩИЙ КОД НЕ РАБОТАЕТ ИЗ-ЗА ТОГО ЧТО В СТОЛБЦЕ "PTS" есть записи DNP и Did Not Play

for file in glob.glob('D:/KAGGLE/Datasets/NBA_1947_2024/NBA/boxscores_by_year/NBA1982/*82_basic.csv'):  
    
    my_df = pd.read_csv(file, engine='python')
    
    my_df_short = my_df[['Game Reference', 'PTS']]                           # СОЗДАЕМ ДФ ТОЛЬКО С НУЖНЫМИ ДАННЫМИ
    
    nonned_df = my_df_short.applymap(lambda x: None if ('DNP') in x else x)  # ЗАПИСЫВАЕМ none если находим текст "DNP"
    new_df = nonned_df.dropna()                                              # УДАЛЯЕМ СТРОКИ С ПУСТЫМИ ЗНАЧЕНИЯМИ
    
    nonned_df = new_df.applymap(lambda x: None if ('Play') in x else x)      # АНАЛОГИЧНО с "PLAY"
    new_df = nonned_df.dropna()
    
    new_df['PTS'] = new_df['PTS'].astype(int)                                # ДЕЛАЕМ СТОЛБЕЦ ЦЕЛОЧИСЛЕННЫМ
        
    filename = file[72] + file[73] + file[74] + file[75]                     # СОЗДАЕМ ПЕРЕМЕННУЮ НАЗВАНИЕ ГОДА ОКОНЧАНИЯ СЕЗОНА
    
    oppo = new_df.groupby('Game Reference', as_index=False).sum()            # СУММИРУЕМ ОЧКИ ПО КАЖДОМУ МАТЧУ
    oppo2 = oppo['PTS'].mean()                                               # ИЩЕМ СРЕДНЕЕ ПО СЕЗОНУ
    
    total_df.loc[len(total_df.index)] = [filename, oppo2]                    # СОХРАНЯЕМ В ДФ


In [190]:
# НАЙДЕМ СРЕДНЕЕ КОЛИЧЕСТВО ОЧКОВ ДЛЯ ФАЙЛОВ папке NBA1983-1996.


for file in glob.glob('D:/KAGGLE/Datasets/NBA_1947_2024/NBA/boxscores_by_year/NBA1983_1996/*_basic.csv'):  
    
    my_df = pd.read_csv(file, engine='python')             # ЧИТАЕМ
    filename = file[77] + file[78] + file[79] + file[80]   # СОЗДАЕМ ПЕРЕМЕННУЮ НАЗВАНИЕ ГОДА ОКОНЧАНИЯ СЕЗОНА
    oppo = my_df.groupby('Game Reference').sum()           # СУММИРУЕМ ОЧКИ С ГРУППИРОВКОЙ ПО КАЖДОМУ МАТЧУ
    oppo2 = oppo['PTS'].mean()                             # ИЩЕМ СРЕДНЕЕ ПО СЕЗОНУ
    total_df.loc[len(total_df.index)] = [filename, oppo2]  # СОХРАНЯЕМ В ДФ


In [194]:
# НАЙДЕМ СРЕДНЕЕ КОЛИЧЕСТВО ОЧКОВ ДЛЯ ФАЙЛОВ в папке NBA1997-2024.


for file in glob.glob('D:/KAGGLE/Datasets/NBA_1947_2024/NBA/boxscores_by_year/NBA1997_2023/*_basic.csv'):  
    
    my_df = pd.read_csv(file, engine='python')              # ЧИТАЕМ
    filename = file[77] + file[78] + file[79] + file[80]    # СОЗДАЕМ ПЕРЕМЕННУЮ НАЗВАНИЕ ГОДА ОКОНЧАНИЯ СЕЗОНА
    oppo = my_df.groupby('Game Reference').sum()            # СУММИРУЕМ ОЧКИ ПО КАЖДОМУ МАТЧУ
    oppo2 = oppo['PTS'].mean() / 3                          # ИЩЕМ СРЕДНЕЕ ПО СЕЗОНУ
   
    total_df.loc[len(total_df.index)] = [filename, oppo2]   # ЗАПИСЫВАЕМ В ДФ


In [198]:
# ИЗМЕНИМ СТРОКОВЫЙ ТИП СТОЛБЦА "YEAR" НА ЦЕЛОЧИСЛЕННЫЙ
total_df['Year'] = total_df['Year'].astype(int) 

In [199]:
total_df.to_csv("NBA_PTS_total_1947_2023.csv")  # СОХРАНЯЕМ В ФАЙЛ